In [10]:
import os

# --- ESTABLECIMIENTO DEL PUNTO DE ORIGEN ---
# Definimos la ruta absoluta del usuario en el contenedor. 
# Esto evita que el código se "pierda" si ejecutas la celda varias veces.
HOME_DIR = "/home/jovyan"

# Forzamos al sistema a situarse en esta carpeta.
# Es como decirle al explorador de archivos: "Vuelve siempre a la carpeta principal".
os.chdir(HOME_DIR)

print(f"✅ Directorio fijado en: {os.getcwd()}")

# --- DEFINICIÓN DE LA ESTRUCTURA DE PROYECTO ---
# Usamos os.path.join para construir rutas de forma segura.
# Esto asegura que las barras (/) funcionen correctamente en cualquier sistema operativo.

# 1. Donde están los archivos .tif originales (entrada)
RAW_DIR = os.path.join(HOME_DIR, "data/raw")

# 2. Donde se guardarán los índices calculados (GeoTIFFs procesados)
PROCESSED_DIR = os.path.join(HOME_DIR, "data/processed")

# 3. Donde se guardarán los mapas finales y gráficos para el informe (PNGs)
FIGURES_DIR = os.path.join(HOME_DIR, "outputs/figures")

# Verificación visual para el usuario
print(f"Ruta para datos: {RAW_DIR}")

✅ Directorio fijado en: /home/jovyan
Ruta para datos: /home/jovyan/data/raw


In [11]:
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt

# --- CONFIGURACIÓN DE RUTAS Y DIRECTORIOS ---
# Definimos dónde están los datos de entrada y dónde guardaremos los resultados
RAW_DIR = "data/raw"              # TIFs originales bajados de Earth Engine
OUT_RASTER_DIR = "data/processed" # Para guardar los índices calculados en formato .tif
OUT_FIG_DIR = "outputs/figures"   # Para guardar los mapas visuales en formato .png

# Creamos las carpetas de salida si no existen para evitar errores
os.makedirs(OUT_RASTER_DIR, exist_ok=True)
os.makedirs(OUT_FIG_DIR, exist_ok=True)

# Lista de años que queremos procesar (deben coincidir con los nombres de archivo)
years = [2017, 2019, 2021, 2024]

# Función auxiliar para evitar errores matemáticos de "división por cero"
# Si b es 0, devuelve NaN (Not a Number), evitando que el script se detenga
def safe_divide(a, b):
    return np.where(b == 0, np.nan, a / b)

# --- PROCESAMIENTO POR AÑO ---
for year in years:
    # Construimos la ruta del archivo original
    path = os.path.join(RAW_DIR, f"sentinel2_pudahuel_{year}.tif")
    
    # Verificamos si el archivo existe antes de intentar abrirlo
    if not os.path.exists(path):
        print(f"Archivo no encontrado para {year}, se omite.")
        continue

    print(f"Procesando {year}...")

    # Abrimos el archivo Raster usando Rasterio
    with rasterio.open(path) as src:
        # Leemos cada banda y la convertimos a float32 para cálculos decimales precisos
        blue  = src.read(1).astype("float32")   # Banda 2 (Azul)
        green = src.read(2).astype("float32")   # Banda 3 (Verde)
        red   = src.read(3).astype("float32")   # Banda 4 (Rojo)
        nir   = src.read(4).astype("float32")   # Banda 8 (Infrarrojo cercano)
        swir1 = src.read(5).astype("float32")   # Banda 11 (Infrarrojo de onda corta 1)
        swir2 = src.read(6).astype("float32")   # Banda 12 (Infrarrojo de onda corta 2)

        # Copiamos los metadatos del original (proyección, coordenadas)
        # Actualizamos para que el archivo de salida sea de 1 sola banda y tipo float32
        meta = src.meta.copy()
        meta.update(count=1, dtype="float32")

    # --- CÁLCULO DE ÍNDICES ESPECTRALES ---
    # NDVI: Identifica vegetación (valores altos = más verde)
    ndvi = safe_divide(nir - red, nir + red)
    
    # NDBI: Identifica áreas urbanas/construidas (valores altos = más cemento)
    ndbi = safe_divide(swir1 - nir, swir1 + nir)
    
    # NDWI: Identifica cuerpos de agua o humedad
    ndwi = safe_divide(green - nir, green + nir)
    
    # BSI: Índice de Suelo Desnudo (útil para ver expansión urbana o erosión)
    bsi  = safe_divide((swir1 + red) - (nir + blue),
                       (swir1 + red) + (nir + blue))

    # Diccionario para iterar y guardar cada índice fácilmente
    indices = {
        "NDVI": ndvi,
        "NDBI": ndbi,
        "NDWI": ndwi,
        "BSI":  bsi
    }

    # --- GUARDADO DE RESULTADOS ---
    for name, data in indices.items():
        # 1. Guardar como archivo GeoTIFF (para análisis espacial posterior en GIS)
        raster_out = os.path.join(
            OUT_RASTER_DIR, f"{name.lower()}_pudahuel_{year}.tif"
        )
        with rasterio.open(raster_out, "w", **meta) as dst:
            dst.write(data, 1)

        # 2. Generar y guardar un mapa visual (Imagen PNG)
        plt.figure(figsize=(6, 5))
        plt.imshow(data, cmap="RdYlGn") # Escala de colores Rojo-Amarillo-Verde
        plt.colorbar(label=name)
        plt.title(f"{name} - Pudahuel {year}")
        plt.axis("off") # Quitar coordenadas de los ejes para que se vea limpio

        fig_out = os.path.join(
            OUT_FIG_DIR, f"{name.lower()}_pudahuel_{year}.png"
        )
        plt.savefig(fig_out, dpi=200, bbox_inches="tight")
        plt.close() # Cerrar la figura para liberar memoria RAM

    print(f"Índices calculados para {year}")

print("Proceso finalizado.")

Procesando 2017...
Índices calculados para 2017
Procesando 2019...
Índices calculados para 2019
Procesando 2021...
Índices calculados para 2021
Procesando 2024...
Índices calculados para 2024
Proceso finalizado.
